# Generate Jobs Report Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### API Request

In [2]:
# Series stored as a dictionary
series = {'LNS14000003': 'White', 
          'LNS14000006': 'Black',
          'LNS14000009': 'Hispanic',
          'LNS14032183': 'Asian',
          'LNS14000000': 'Total',
          'LNS13327709': 'U6',
          'LNS13000000': 'Level',
          'LNU05026639': 'WantJob',
          'LNU03008636': 'LT',
          'LNU03008516': 'MT',
          'LNU00000000': 'POP',
          'LNS12300060': 'PA_EPOP',
          'LNS13023621': 'Job Loser',
          'LNS13023653': 'Temporary Layoff',
          'LNS13026638': 'Permanent Separation',
          'LNS13023705': 'Job Leaver', 
          'LNS13023557': 'Re-entrant',
          'LNS13023569': 'New entrant',
          'LNS13008276': 'Median',
          'LNS13008275': 'Mean',
          'LNS17200000': 'NILF',
          'LNS17100000': 'UNEMP',
          'LNS11000000': 'LF',
          'LNS12032194': 'PTECON',
          'LNS12005977': 'PTNONECON'}

# Start year and end year
dates = (1988, 2023)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'jobs_report_main.csv', index_label='date')
print(dtxt(df.index[-1])['mon1'])

Post Request Status: REQUEST_SUCCEEDED
December 2023


In [3]:
# Series stored as a dictionary
series = {'LNS12300061': 'PA_EPOP_M', 
          'LNS12300062': 'PA_EPOP_W',
          'LNS12005054': 'avghrstot',
          'LNU02033699': 'avghrsserv',
          'LNU02033232': 'avghrsptecon',
          'LNU01000000': 'LFnsa',
          'LNS12026619': 'MJHsa',
          'LNU02000000': 'EMP',
          'LNS12000000': 'EMPsa',
          'LNU00000001': 'MenPop',
          'LNU00000002': 'WomenPop',
          'LNU01000001': 'MenLF',
          'LNU01000002': 'WomenLF',
          'LNS11300001': 'MenLFPR',
          'LNS11300002': 'WomenLFPR',
          'LNU02048984': 'seinc',
          'LNS12027714': 'seuninc',
          'LNU02374597': 'empdis',
          'LNS12300000': 'EPOP'}

# Start year and end year
dates = (1988, 2023)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'jobs_report_main2.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [4]:
# Series stored as a dictionary
series = {'LNS17100001': 'MenUE',
          'LNS17100002': 'WomenUE',
          'LNS17200001': 'MenNE',
          'LNS17200002': 'WomenNE',
          'LNS17400001': 'MenEU',
          'LNS17400002': 'WomenEU',
          'LNS17600001': 'MenNU',
          'LNS17600002': 'WomenNU',
          'LNS17800001': 'MenEN',
          'LNS17800002': 'WomenEN',
          'LNS17900001': 'MenUN',
          'LNS17900002': 'WomenUN',
          'LNS12000001': 'MenE',
          'LNS12000002': 'WomenE',
          'LNS13000001': 'MenU',
          'LNS13000002': 'WomenU',
          'LNS15000001': 'MenN',
          'LNS15000002': 'WomenN'}

# Start year and end year
dates = (2018, 2023)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'jobs_report_main3.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### Labor Force Gross Flows

In [2]:
df = (pd.read_csv(data_dir / 'jobs_report_main3.csv', parse_dates=['date'])
        .set_index('date')) / 1000

cols = ['MenEU', 'WomenEU', 'MenEN', 'WomenEN', 'MenUE', 'WomenUE',
        'MenUN', 'WomenUN', 'MenNE', 'WomenNE', 'MenNU', 'WomenNU']

cols2 = []
for col in cols:
    name = f'{col}{col[-2]}'
    cols2.append(name)
    df[name] = (df[col] / df[f'{col[:-2]}{col[-2]}'].shift()) * 100

df.loc['2013-01-01':, cols2].to_csv(data_dir / 'grosslf.csv', index_label='date')

### Unemployment rate

In [3]:
df = pd.read_csv(data_dir / 'jobs_report_main.csv', index_col='date', 
                 parse_dates=True)
srs = ['Total', 'U6']
df.loc['1989':, srs].to_csv(data_dir / 'unemp2.csv', index_label='date')

srs = ['White', 'Black', 'Hispanic']
df.loc['1989':, srs].to_csv(data_dir / 'unemp.csv', index_label='date')

s = series_info(df['Level'])
s2 = series_info(df['Total'])
s3 = series_info(df['Black'])
s4 = series_info(df['U6'])
compare = compare_text(df['Total'].iloc[-1], df['Total'].iloc[-2], [0.15, 1.5, 3.0])
compare2 = compare_text(df['Total'].iloc[-1], df['Total'].iloc[-13], [0.15, 1.5, 3.0])
pryrdt = dtxt(df.index[-13])['mon1']

if compare[-5:] != compare2[-5:]:
    conj = f', but {compare2} the {pryrdt} rate of {df["Total"].iloc[-13]:.1f} percent'
elif compare != compare2:
    conj = f', and {compare2} the {pryrdt} rate of {df["Total"].iloc[-13]:.1f} percent'
else:
    conj = ''
    
text = ('BLS \href{https://www.bls.gov/news.release/empsit.nr0.htm}{reports} '+
        f'{s["val_latest"]/1000:.1f} million '+
        f'unemployed people in {s["date_latest_ft"]}, '+
        f'and an unemployment rate of {s2["val_latest"]} percent '+
        '(see {\color{blue!50!cyan}\\textbf{---}}), '+
        f'{compare} the {s["date_prev_ft"]} rate of {s2["val_prev"]} percent'+
        f'{conj}.')
write_txt(text_dir / 'unemp1.txt', text)
print(text, '\n')

mval = f', {s4["last_matched"]}.' if s4['days_since_match'] > 1000 else '.'
text = (f'In {s["date_latest_ft"]}, the labor under-utilization rate is '+
        f'{s4["val_latest"]} percent '+
        '(see {\color{blue}\\textbf{---}})'+
        f'{mval}')
write_txt(text_dir / 'unemp2.txt', text)
print(text, '\n')

write_txt(text_dir / 'u6_node.txt', 
          end_node(df['U6'], 'blue', date='m', offset=0.35))
write_txt(text_dir / 'u3_node.txt', end_node(df['Total'], 'blue!50!cyan'))

black_ch = df['Black'].iloc[-1] - df.loc['2020-02-01', 'Black']
bch = value_text(black_ch, style='increase_by', ptype='pp')
text = ('Periods of unemployment are more common for disadvantaged groups. '+
        'The black or African American unemployment rate is typically '+
        'double the white unemployment rate. Employment opportunities for '+
        'disadvantaged groups are more-dependent on current labor market '+
        'conditions. A very tight labor market reduces racial '+
        'discrimination in hiring, while disadvantaged groups are more '+
        'likely to lose jobs in a downturn. '+
        'Since February 2020, the black unemployment rate '+
        f'has {bch} to {s3["val_latest"]:.1f} percent '+
        '(see {\color{green!50!teal!60!black}\\textbf{---}}).')
write_txt(text_dir / 'unemp3.txt', text)
print(text)

BLS \href{https://www.bls.gov/news.release/empsit.nr0.htm}{reports} 6.3 million unemployed people in December 2023, and an unemployment rate of 3.7 percent (see {\color{blue!50!cyan}\textbf{---}}), in line with the November 2023 rate of 3.7 percent, but slightly above the December 2022 rate of 3.5 percent. 

In December 2023, the labor under-utilization rate is 7.1 percent (see {\color{blue}\textbf{---}}). 

Periods of unemployment are more common for disadvantaged groups. The black or African American unemployment rate is typically double the white unemployment rate. Employment opportunities for disadvantaged groups are more-dependent on current labor market conditions. A very tight labor market reduces racial discrimination in hiring, while disadvantaged groups are more likely to lose jobs in a downturn. Since February 2020, the black unemployment rate has decreased by 0.9 percentage point to 5.2 percent (see {\color{green!50!teal!60!black}\textbf{---}}).


In [4]:
srs = ['U6', 'Total', 'White', 'Black', 'Hispanic', 'Asian']
untab = df[srs].iloc[[-1, -2, -3, -4, -13, -25]].T
untab.columns = untab.columns.strftime('%b `%y')
untab['GFC peak'] = df.loc['2005':'2013', srs].max()
untab['Date of peak'] = df.loc['2005':'2013', srs].idxmax().dt.strftime('%b `%y')
d = {'Total': 'Unemployment Rate (U3)',
     'U6': 'Under-utilization Rate (U6)',
     'White': '\hspace{2mm} White',
     'Black': '\hspace{2mm} Black',
     'Hispanic': '\hspace{2mm} Hispanic',
     'Asian': '\hspace{2mm} Asian'}
untab.index = untab.index.map(d)

untab.loc['\\textit{by race/ethnicity:}', untab.columns] = [''] * 8
untab = pd.concat([untab.iloc[0:2], untab.iloc[-1].to_frame().T, untab.iloc[2:6]])
untab.columns.name = None
untab.to_csv(data_dir / 'unemp1.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

untab

,Dec `23,Nov `23,Oct `23,Sep `23,Dec `22,Dec `21,GFC peak,Date of peak
Under-utilization Rate (U6),7.1,7.0,7.2,7.0,6.5,7.3,17.2,Dec `09
Unemployment Rate (U3),3.7,3.7,3.8,3.8,3.5,3.9,10.0,Oct `09
\textit{by race/ethnicity:},,,,,,,,
\hspace{2mm} White,3.5,3.3,3.5,3.4,3.0,3.3,9.2,Oct `09
\hspace{2mm} Black,5.2,5.8,5.8,5.7,5.7,6.9,16.8,Mar `10
\hspace{2mm} Hispanic,5.0,4.6,4.8,4.6,4.2,4.6,13.0,Aug `09
\hspace{2mm} Asian,3.1,3.5,3.1,2.9,2.4,3.8,8.4,Dec `09


### Labor Force Participation Rate

In [5]:
df = (pd.read_csv(data_dir / 'jobs_report_main2.csv', parse_dates=['date'])
        .set_index('date'))[['MenLFPR', 'WomenLFPR']]
df['TotLFPR'] = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                             parse_dates=['date'])
                   .assign(TotLFPR = lambda x: (x.LF / x.POP)*100)
                   .set_index('date'))['TotLFPR']
df.loc['1989':].to_csv(data_dir / 'lfpr.csv', index_label='date')

col = {'MenLFPR': 'blue!70!black',
       'WomenLFPR': 'red!80!black',
       'TotLFPR': 'green!80!blue!60!black'}
nodes = (end_node(df['MenLFPR'], col['MenLFPR'], date='m', 
                  offset=0.35) + '\n' + 
         '\n'.join(end_node(df[name], color) 
                   for name, color in col.items() if name != 'MenLFPR'))
write_txt(text_dir / 'lfpr_nodes.txt', nodes)

tot = df['TotLFPR']
ltdt = dtxt(df.index[-1])['mon1']
write_txt(text_dir / 'lfpr_blsdate.txt', ltdt)
prdt, prdt2 = (dtxt(df.index[i])['mon3'] 
               if df.index[-1].year == df.index[i].year 
               else dtxt(df.index[i])['mon1'] 
               for i in [-2, -3])
cpdt = '2020-02-01'
compdt = dtxt(cpdt)['mon1']
feb20val = tot.loc[cpdt]
mltval = df['MenLFPR'].iloc[-1]
wltval = df['WomenLFPR'].iloc[-1]
mchval = mltval - df['MenLFPR'].loc['2020-01-01']
wchval = wltval - df['WomenLFPR'].loc['2020-01-01']
mch = value_text(mchval, style='increase', ptype='pp')
wch = value_text(wchval, style='increase', ptype='pp')
cl = {k: c_line(v) for k, v in col.items()}
v1, v2, v3 = [value_text(tot.iloc[i], 'plain') for i in [-1, -2, -3]]
chtxt = 'following'
comptxt = (f', {chtxt} {v2} in {prdt} and {v3} in {prdt2}')
if v1 == v2 == v3:
    comptxt = (f', the same as in {prdt} and {prdt2}')
elif v1 != v2 != v3:
    comptxt = comptxt
else:
    print('Partial match \n')
    if v1 == v2:
        comptxt = (f', unchanged from {prdt}, and following {v3} in {prdt2}')
    if v1 == v3:
        comptxt = comptxt
    if v2 == v3:
        comptxt = (f', {chtxt} {v2} in both {prdt} and {prdt2}')
text = (f'As of {ltdt}, {v1} of those aged 16 and over are part '+
        f'of the labor force {cl["TotLFPR"]}{comptxt}. Pre-pandemic, '+
        f'in {compdt}, the rate stood at {feb20val:.1f} percent.\n\n'+
        f'In {ltdt}, {mltval:.1f} percent of men age '+
        f'16 and older are in the labor force {cl["MenLFPR"]}, compared to '+
        f'{wltval:.1f} percent of women {cl["WomenLFPR"]}. Since '+
        f'{compdt}, labor force participation has {mch} among men, '+
        f'and {wch} among women.')
write_txt(text_dir / 'lfpr_text.txt', text)
print(text)

As of December 2023, 62.5 percent of those aged 16 and over are part of the labor force (see {\color{green!80!blue!60!black}\textbf{---}}), following 62.8 percent in November and 62.7 percent in October. Pre-pandemic, in February 2020, the rate stood at 63.3 percent.

In December 2023, 68.1 percent of men age 16 and older are in the labor force (see {\color{blue!70!black}\textbf{---}}), compared to 57.1 percent of women (see {\color{red!80!black}\textbf{---}}). Since February 2020, labor force participation has decreased 1.1 percentage points among men, and decreased 0.8 percentage point among women.


### Employment rate

In [6]:
url = 'https://data.bls.gov/timeseries/LNS12300000'
pa_url = 'https://data.bls.gov/timeseries/LNS12300060'
df1 = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  index_col='date', parse_dates=True)
        .loc['1989':, 'PA_EPOP'])
df2 = (pd.read_csv(data_dir / 'jobs_report_main2.csv', 
                  index_col='date', parse_dates=True)
        .loc['1989':, 'EPOP'])
df = pd.concat([df1, df2], axis=1)
df.to_csv(data_dir / 'epop.csv', index_label='date')

color = 'green!60!black'
node = end_node(df['EPOP'], color, date='m')
write_txt(text_dir / 'epop_node2.txt', node)

ltdt = dtxt(df.index[-1])['mon1']
ltval = df['EPOP'].iloc[-1]
chval = df['EPOP'].diff(12).iloc[-1]
chtxt = value_text(chval, 'increase_of', ptype='pp', time_str='one-year ')
chval2 = df['EPOP'].iloc[-1] - df.loc['2019', 'EPOP'].mean()
chtxt2 = value_text(chval2, style='increase_end', ptype='pp')
ab = 'but'
if ((chval<0) == (chval2<0)) == True:
    ab = 'and'
text = (f'As of {ltdt}, the Bureau of Labor Statistics '+
        f'\href{{{url}}}{{report}} '+
        f'an overall (age 16 and older) employment rate of {ltval} '+
        f'percent {c_line(color)}, {chtxt}, {ab} {chtxt2} since 2019.')
write_txt(text_dir / 'epop_text2.txt', text)
print(text)

color = 'blue!90!cyan'
adj = 0
if (df['PA_EPOP'].iloc[-1] / df['PA_EPOP'].max()) > 0.95:
    adj = -0.25
elif (df['PA_EPOP'].iloc[-1] / df['PA_EPOP'].max()) > 1:
    adj = -0.4
node = end_node(df['PA_EPOP'], color, date='m', offset=adj)
write_txt(text_dir / 'epop_node.txt', node)

ltval = df['PA_EPOP'].iloc[-1]
prval = df['PA_EPOP'].iloc[-2]
prdt = dtxt(df['PA_EPOP'].index[-2])['mon1']
prtxt = f'compared to {prval} percent in {prdt}'
compval = df['PA_EPOP'].loc['2019-06-01': '2020-03-01'].max()
last = series_info(df['PA_EPOP'])['last_matched']
text2 = prtxt if ltval < compval else last
chtxt = value_text(df['PA_EPOP'].diff(12).iloc[-1], ptype='pp', threshold=0.1)

pop = (cps_1mo(cps_dir, cps_date(), ['BASICWGT', 'AGE'])
       .query('25 <= AGE <=54').BASICWGT.sum()) / 1_000
rt99 = df['PA_EPOP'].loc['1999': '2000'].mean()
ch99 =  ltval - rt99
ch99w = (-ch99 / 100) * pop
ch99t = (f'{round(abs(ch99w) / 1000, 1)} million' 
         if ch99w > 999 else f'{round(abs(ch99w), -1):.0f},000')
ch99t2 = f'(equivalent to {ch99t} workers)'
threshold = 0.1
txt = value_text(ch99, style='above_below', 
                 ptype='pp', threshold=threshold)
if abs(ch99) > threshold:
    ch99txt = f'{txt[:-6]} {ch99t2} {txt[-5:]}'
else:
    ch99txt = txt
text = (f'In {ltdt}, {ltval} percent of 25 to 54 years olds were '+
        f'employed {c_line(color)}, {text2}. Over the past year, '+
        f'the age 25 to 54 employment rate {chtxt}. The {ltdt} rate '+
        f'is {ch99txt} the average rate of {rt99:.1f} during the '+
        'tight labor market of 1999--2000.')
write_txt(text_dir / 'epop_text.txt', text)
print(text)

As of December 2023, the Bureau of Labor Statistics \href{https://data.bls.gov/timeseries/LNS12300000}{report} an overall (age 16 and older) employment rate of 60.1 percent (see {\color{green!60!black}\textbf{---}}), a one-year increase of 0.0 percentage point, but a 0.7 percentage point decrease since 2019.
In December 2023, 80.4 percent of 25 to 54 years olds were employed (see {\color{blue!90!cyan}\textbf{---}}), compared to 80.7 percent in November 2023. Over the past year, the age 25 to 54 employment rate increased 0.3 percentage point. The December 2023 rate is one percentage point (equivalent to 1.3 million workers) below the average rate of 81.4 during the tight labor market of 1999--2000.


### Recent Values

In [7]:
df1 = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  index_col='date', parse_dates=True)
        .loc['1989':, 'PA_EPOP'])
df2 = (pd.read_csv(data_dir / 'jobs_report_main2.csv', 
                  index_col='date', parse_dates=True)
        .loc['1989':, ['PA_EPOP_M', 'PA_EPOP_W']])
df = pd.concat([df1, df2], axis=1)
df['label'] = [dt.strftime('%b\\\%Y') if dt.month == 1 
                  else dt.strftime('%b') if dt.month == 7
                  else '' for dt in df.index]
res = df.iloc[-25:]

res.to_csv(data_dir / 'pa_epop_rec.csv', index_label='date')

In [8]:
sdt = res.index[0]
sdtime = (dtxt(sdt)['datetime']
           .replace('-08-', '-8-').replace('-09-', '-9-'))
ltdt = res.index[-1]
ltdtime = (dtxt(ltdt)['datetime']
           .replace('-08-', '-8-').replace('-09-', '-9-'))
node = []
d = {'PA_EPOP': ['blue!80!purple', 'cyan'], 
     'PA_EPOP_M': ['green!50!black', 'green!50!black'], 
     'PA_EPOP_W': ['yellow!40!orange', 'yellow!40!orange']}
for s, [c1, c2] in d.items():
    dt = f'\scriptsize {dtxt(ltdt)["mon6"]} \\\\' if s == 'PA_EPOP_M' else ''
    v0 = res[s].iloc[0]
    vlt = res[s].iloc[-1]
    y = 0.15 if s == 'PA_EPOP_M' else '0'
    x = -0.35 if s == 'PA_EPOP_M' else '-0.05'
    n0 = ('\\node[label={[yshift=0cm, xshift=0.05cm, align=right]180:'+
          f'{{\scriptsize {v0:.1f}}}}}, circle, draw={c1}, fill={c2}, '+
          f'inner sep=1pt] at (axis cs:{sdtime}, {v0}) '+
          '{};\n'+
          f'\\node[label={{[yshift={y}cm, xshift={x}cm, align=right]0:'+
          f'{{{dt}\scriptsize {vlt:.1f}}}}}, circle, draw={c1}, fill={c2}, '+
          f'inner sep=1pt] at (axis cs:{ltdtime}, {vlt}) '+
          '{};')
    node.append(n0)
    
# Text labels
k = [['south east', 'right', dtxt(res.index[10])['datetime2'], 
  res['PA_EPOP_M'].iloc[10], 'green!50!black', '\\footnotesize Men'], 
 ['north west', 'left', dtxt(res.index[4])['datetime2'], 
  res['PA_EPOP_W'].iloc[4], 'yellow!40!orange', '\\footnotesize Women'], 
 ['south east', 'left', dtxt(res.index[15])['datetime2'], 
  res['PA_EPOP'].iloc[15], 'blue!80!cyan!90!white', '\small Total']]

tnode = [(f'\\node[anchor={anchor}, align={align}] at '+
         f'({{{dt}}}, {val}) {{\color{{{color}}}{name}}};') 
         for anchor, align, dt, val, color, name in k]

nodes = '\n'.join([n for n in [*node, *tnode]])
write_txt(text_dir / 'epop_summary_nodes.txt', nodes)

### Unemployment by reason

In [9]:
srs = ['Job Loser', 'Job Leaver', 'Re-entrant', 'New entrant', 
       'Temporary Layoff', 'Permanent Separation', 'Level']
d1 = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  parse_dates=['date'])
        .set_index('date')).loc['1989':]

df = d1[srs].div(d1['LF'], axis='index') * 100
dfa = df.resample('AS').mean()
dfa.index = dfa.index + pd.DateOffset(months=6)
dfa.to_csv(data_dir / 'unemp_reason.csv', index_label='date', 
          float_format='%g')

lt = df.iloc[-3:]
lt.index = [dtxt(i)['mon7'] for i in lt.index]
lt = pd.concat([df.rolling(12).mean().iloc[-1].rename('12m avg'), lt.T], axis=1).T
lt.to_csv(data_dir / 'unemp_reason_mon.csv', index_label='date', 
          float_format='%g')

x_val = dtxt(dfa.index[-1])['datetime']
x_valpr = dtxt(dfa.index[-2])['datetime']
y_val = dfa.Level.iloc[-1]
prelim = '(p)' if df.index[-1].month != 12 else ''
yrp = f'{dtxt(dfa.index[-1])["year"][2:]}{prelim}'
barh = ('\\addplot[ybar, bar width=5.4pt, draw=black, fill=white!0] '+
        f'plot coordinates{{({x_val},{y_val})}};\n'+
        f'\\absnode{{{x_valpr}}}{{-0.25}}{{\scriptsize \color{{black!70}} {yrp}}};')
write_txt(text_dir / 'unemp_rsn_ltbar.txt', barh)

cols = ['Job Loser', 'New entrant', 'Re-entrant', 'Job Leaver']
sdf = lt[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 0.25)).to_dict()
val = sdf.to_dict()
nodes = [f'\\absnode{{3.3}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' for i in cols]
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'unemp_rsn_ltval.txt', nodetext)

colors = {'Job Loser': 'red!75!orange!70!white', 'New entrant': 'purple!80!red!85!black', 
          'Re-entrant': 'blue!70!teal!70!gray!62!white', 'Job Leaver': 'blue!72!black'}
cl = {k: c_box(v) for k, v in colors.items()}

ltdt = dtxt(df.index[-1])['mon1']
lt = df.iloc[-1]
jlsh = lt['Job Loser']
jl = (d1['Job Loser'].iloc[-1] / 1_000)
jle = lt['Job Leaver']
re = lt['Re-entrant']
ne = lt['New entrant']
text = ('There are several \\textbf{reasons for unemployment}. In '+
        f'{ltdt}, {jl:.1f} million people, or {jlsh:.1f} percent of '+
        'the labor force, were unemployed from losing their job '+
        f'{cl["Job Loser"]}. An additional {jle:.1f} percent '+
        f'voluntarily left a job {cl["Job Leaver"]}. Re-entrants, '+
        'people who left the labor force but are looking for a new '+
        f'job, comprised {re:.1f} percent {cl["Re-entrant"]}. Lastly, '+
        f'{ne:.1f} percent of the labor force were new entrants to '+
        f'the labor market, looking for their first job '+
        f'{cl["New entrant"]}.')
write_txt(text_dir / 'unemp_rsn.txt', text)
print(text)

lf = ['Employed', 'Unemployed']
naw_rate = lambda x: np.average(x['NOTATWORK'], weights=x['BASICWGT'])

columns = ['LFS', 'MONTH', 'YEAR', 'BASICWGT', 'NOTATWORK']

naw = (pd.concat([(pd.read_feather(cps_dir / f'cps{year}.ft', columns=columns)
              .query('LFS in @lf'))
           for year in range(2009, 2024)])
        .groupby(['YEAR', 'MONTH'])
        .apply(naw_rate) * 100)
naw.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in naw.index]
df['Employed, Not at Work'] = naw

tbl = df.iloc[-3:].T.iloc[:, ::-1]
tbl.columns = [dtxt(i)['mon2'] for i in tbl.columns]
tbl['12m Avg.'] = df.rolling(12).mean().iloc[-1]
tbl['Apr 2020'] = df.loc['2020-04-01']
tbl['2020'] = df.loc['2020'].mean()
tbl['2019'] = df.loc['2019'].mean()
tbl['2009 --`11'] = df.loc['2009': '2011'].mean()
tbl = tbl.round(1)
d = {'Level': '\ Unemployed, Any Reason',
     'Job Loser': f'\hspace{{2mm}}\cbox{{{colors["Job Loser"]}}} Job Loser',
     'Temporary Layoff': '\hspace{9mm}Temporary Layoff',
     'Permanent Separation': '\hspace{9mm}Permanent Separation',
     'Re-entrant': f'\hspace{{2mm}}\cbox{{{colors["Re-entrant"]}}} Re-entrant',
     'New entrant': f'\hspace{{2mm}}\cbox{{{colors["New entrant"]}}} New entrant',
     'Job Leaver': f'\hspace{{2mm}}\cbox{{{colors["Job Leaver"]}}} Job Leaver'}

final = tbl.loc[d.keys()].rename(d)

final.loc['\\textit{See also:}', final.columns] = [''] * 8
final.loc['\ \ Employed, Not at Work*'] = tbl.loc['Employed, Not at Work']
final.to_csv(data_dir / 'unempreason_table.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

tl = lt['Temporary Layoff']
pjl = lt['Permanent Separation']

text = (f'In {ltdt}, temporary layoffs were {tl:.1f} percent of '+
        f'the labor force. Permanent job losses were {pjl:.1f} '+
        'percent of labor force. ')
write_txt(text_dir / 'unemp_rsn2.txt', text)
print('\n', text)

There are several \textbf{reasons for unemployment}. In December 2023, 3.1 million people, or 1.8 percent of the labor force, were unemployed from losing their job (see\cbox{red!75!orange!70!white}). An additional 0.5 percent voluntarily left a job (see\cbox{blue!72!black}). Re-entrants, people who left the labor force but are looking for a new job, comprised 1.0 percent (see\cbox{blue!70!teal!70!gray!62!white}). Lastly, 0.4 percent of the labor force were new entrants to the labor market, looking for their first job (see\cbox{purple!80!red!85!black}).

 In December 2023, temporary layoffs were 0.5 percent of the labor force. Permanent job losses were 0.9 percent of labor force. 


### Unemployed long-term

In [10]:
srs = ['LT', 'MT', 'POP']
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', parse_dates=['date'])
        .set_index('date')).loc['1989':, srs]

data = (df.divide(df['POP'], axis=0) * 100).drop(['POP'], axis=1)
data.to_csv(data_dir / 'ltu.csv', index_label='date', float_format='%g')

collt = 'blue'
node = end_node(data['LT'], collt, date='m', digits=2, offset=0.35)
write_txt(text_dir / 'ltu_node.txt', node)
colmt = 'violet!90!black'
node = end_node(data['MT'], colmt, digits=2, date='m', offset=0.35)
write_txt(text_dir / 'ltu_node2.txt', node)

dates = date_list(data)
pydt = dtxt(data.index[-13])['mon1']
mxdt = dtxt(data['LT'].idxmax())['mon1']
vl1 = value_text(data["LT"].iloc[-1], 'plain', digits=2)
v1 = value_text(data["MT"].iloc[-1], 'plain', digits=2)
v2 = value_text(data["MT"].iloc[-2], 'plain', digits=2)
v3 = value_text(data["MT"].iloc[-3], 'plain', digits=2)
comptxt = (f'{v1} in {dates[1]} and {v2} in {dates[2]}.')
if v2 == v3:
    comptxt = (f'{v2} in both {dates[1]} and {dates[2]}')

cllt = c_line(collt)
clmt = c_line(colmt)
recent_min = data.loc['2015':'2020-02-01', 'LT'].min()
recent_min_dt = dtxt(data.loc['2015':'2020-02-01', 'LT'].idxmin())['mon1']
url = 'https://www.bls.gov/webapps/legacy/cpsatab12.htm'
text = (f'As of {dates[0]}, BLS \href{{{url}}}{{reports}} that '+
        f'{vl1} of the age 16 and older population have been '+
         'unemployed for 27 weeks or longer, compared to '+
        f'{data["LT"].iloc[-13]:.2f} percent in {pydt} {cllt}. '+
        'This measure of \\textbf{long-term unemployment} peaked '+
        f'at {data["LT"].max():.2f} percent of the population '+
        f'in {mxdt}, but had fallen to {recent_min:.2f} percent '+
        f'in {recent_min_dt}.\n\n In {dates[0]}, {v1} of those '+
        'age 16 and older have been unemployed for at least 15 '+
        f'weeks {clmt}, following {comptxt}. One-year prior, in '+
        f'{pydt}, {data["MT"].iloc[-13]:.2f} percent are unemployed '+
        'for 15 weeks or more. ')
write_txt(text_dir / 'ltu.txt', text)
print(text)

As of December 2023, BLS \href{https://www.bls.gov/webapps/legacy/cpsatab12.htm}{reports} that 0.44 percent of the age 16 and older population have been unemployed for 27 weeks or longer, compared to 0.37 percent in December 2022 (see {\color{blue}\textbf{---}}). This measure of \textbf{long-term unemployment} peaked at 2.96 percent of the population in April 2010, but had fallen to 0.42 percent in December 2019.

 In December 2023, 0.82 percent of those age 16 and older have been unemployed for at least 15 weeks (see {\color{violet!90!black}\textbf{---}}), following 0.82 percent in November and 0.75 percent in October.. One-year prior, in December 2022, 0.65 percent are unemployed for 15 weeks or more. 


### Duration of Unemployment

In [11]:
s = {'Median': 'green!75!blue', 'Mean': 'blue!60!cyan'}
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  parse_dates=['date'])
        .set_index('date')).loc['1989':, s.keys()]
df.to_csv(data_dir / 'unempdur.csv', index_label='date', 
          float_format='%g')
ltdt = dtxt(df.index[-1])['mon1']
lt = df.iloc[-1]

adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

date = {series: 'm' if series == smax else None 
        for series in s.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in s.items()])
write_txt(text_dir / 'unempdur_nodes.txt', nodes) 

pc_yr = df.loc['2019-03-01':'2020-02-01'].mean()
text = (f'Among those who are unemployed in {ltdt}, the average '+
        f'(mean) \\textbf{{duration of unemployment}} is {lt.Mean:.1f} '+
        f'weeks {c_line(s["Mean"])}, and the typical (median) '+
        f'duration of unemployment is {lt.Median:.1f} weeks '+
        f'{c_line(s["Median"])}. Over the year '+
        'prior to COVID-19, ending February 2020, the '+
        'average duration of unemployment was '+
        f'{pc_yr.Mean:.1f} weeks and the typical '+
        f'duration was {pc_yr.Median:.1f} weeks.')
write_txt(text_dir / 'unempdur.txt', text)
print(text)

Among those who are unemployed in December 2023, the average (mean) \textbf{duration of unemployment} is 22.3 weeks (see {\color{blue!60!cyan}\textbf{---}}), and the typical (median) duration of unemployment is 9.7 weeks (see {\color{green!75!blue}\textbf{---}}). Over the year prior to COVID-19, ending February 2020, the average duration of unemployment was 21.7 weeks and the typical duration was 9.2 weeks.


### Part-Time Workers

In [12]:
srs = ['PTECON', 'PTNONECON', 'LF']
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  index_col='date', parse_dates=True)
        .loc['1989':, srs])
df['TOTAL'] = df['PTECON'] + df['PTNONECON']
# Total part-time for summary text
text = (f'Around {df.TOTAL.iloc[-1] / 1000:.0f} million people '+
        'work part-time, defined as fewer than 35 hours per week, '+
        'and the reasons for doing so vary.')
write_txt(text_dir / 'pt_tot.txt', text)
print(text, '\n')

# Share of labor force
data = (df.divide(df.LF, axis=0) * 100)
data = data.loc['1994':, ['PTECON', 'PTNONECON', 'TOTAL']]
data.to_csv(data_dir / 'parttime.csv', index_label='date')

color = 'red'
col2 = 'orange!80!yellow'
node = end_node(data.PTECON, color)
node2 = end_node(data.PTNONECON, col2, 
                 date='m', offset=-0.3)
nodes = node + '\n' + node2
write_txt(text_dir / 'pt_nodes.txt', nodes)

ltdt = dtxt(df.index[-1])['mon1']
lttot = df.PTECON.iloc[-1] / 1_000
ltvpt = df.PTNONECON.iloc[-1] / 1_000
ltsh = data.PTECON.iloc[-1]
val19 = data.loc['2019', 'PTECON'].mean()
val10 = data.loc['2010', 'PTECON'].mean()
ltvsh = value_text(data.PTNONECON.iloc[-1], 'plain')
vsh19 = value_text(data.loc['2019', 'PTNONECON'].mean(), 'plain')
also = 'also ' if ltvsh == vsh19 else ''

text = (f'In {ltdt}, {lttot:.1f} million people worked part-time '+
        f'for economic reasons, equivalent to {ltsh:.1f} percent '+
        f'of the labor force {c_line(color)}. In 2019, an average '+
        f'of {val19:.1f} percent of the labor force worked part-time '+
        'for economic reasons. In 2010, following the great '+
        f'recession, the rate was {val10:.1f} percent.\n\nVoluntary '+
        f'part-time workers total {ltvpt:.1f} million in {ltdt}, or '+
        f'{ltvsh} of the labor force {c_line(col2)}. The '+
        f'category is {also}{vsh19} of the labor force in '+
        '2019, on average.')
write_txt(text_dir / 'parttime.txt', text)
print(text)

Around 27 million people work part-time, defined as fewer than 35 hours per week, and the reasons for doing so vary. 

In December 2023, 4.2 million people worked part-time for economic reasons, equivalent to 2.5 percent of the labor force (see {\color{red}\textbf{---}}). In 2019, an average of 2.7 percent of the labor force worked part-time for economic reasons. In 2010, following the great recession, the rate was 5.8 percent.

Voluntary part-time workers total 22.5 million in December 2023, or 13.4 percent of the labor force (see {\color{orange!80!yellow}\textbf{---}}). The category is 13.1 percent of the labor force in 2019, on average.


### Multiple Jobholders

In [13]:
df = (pd.read_csv(data_dir / 'jobs_report_main2.csv', 
                  index_col='date', parse_dates=True)
        .loc['1989':, ['MJHsa', 'EMPsa']])
data = ((df.MJHsa / df.EMPsa) * 100).dropna().rename('MJH')
d3m = data.rolling(3).mean()
d3m.to_csv(data_dir / 'mjh.csv', index_label='date')
color = 'blue!50!violet!80!black'
node = end_node(d3m, color, size=1.2, date='m')
write_txt(text_dir / 'mjh_node.txt', node)

ltdt = dtxt(df.index[-1])['mon1']
totval = df.MJHsa.iloc[-1] / 1_000
ltval = value_text(data.iloc[-1], 'plain')
l3val = f'{d3m.iloc[-1]:.1f}'
l19val = data.loc['2019'].mean()
comp_date = '2020-02-01'
same = 'Over' # Less awkward text when latest and 3ma match
if ltval == l3val:
    same = 'Likewise, over'

text = (f'In {ltdt}, a seasonally-adjusted total of {totval:.1f} '+
        'million people \\textbf{worked '+
        'more than one job} during the survey reference week, '+
        f'equivalent to {ltval} of workers. '+
        f'{same} the three months ending {ltdt}, an average of '+
        f'{l3val} percent of workers were multiple '
        f'jobholders {c_line(color)}. In 2019, an average of '+
        f'{l19val:.1f} percent of workers had more than one job '+
        'during the survey reference week. ')
write_txt(text_dir / 'mjh.txt', text)
print(text)

In December 2023, a seasonally-adjusted total of 8.6 million people \textbf{worked more than one job} during the survey reference week, equivalent to 5.3 percent of workers. Over the three months ending December 2023, an average of 5.2 percent of workers were multiple jobholders (see {\color{blue!50!violet!80!black}\textbf{---}}). In 2019, an average of 5.1 percent of workers had more than one job during the survey reference week. 


### Self-Employed

In [14]:
df = (pd.read_csv(data_dir / 'jobs_report_main2.csv', 
                  index_col='date', parse_dates=True)
        .loc['1988':, ['seinc', 'seuninc', 'LFnsa']])

df2 = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  index_col='date', parse_dates=True)
        .loc['1988':, ['LF']])

df3 = pd.read_csv(data_dir / 'se_inc_hist.csv', index_col='date', 
                  parse_dates=True)

uninc = (df['seuninc'].divide(df2['LF'], axis=0) * 100).rename('uninc')
inc = (df['seinc'].divide(df['LFnsa'], axis=0) * 100).rename('inc')
INC = (df3.INC.divide(df['LFnsa'], axis=0) / 10).rename('INC')
res = uninc.to_frame().join(inc).join(INC).loc['1989':]
res.to_csv(data_dir / 'selfemp.csv', index_label='date')
dft = res[['inc', 'uninc']]

adj = node_adj(dft)
smax = dft.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'inc': 'green!80!black', 
          'uninc': 'purple'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(dft[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'selfemp_nodes.txt', nodes)  

In [15]:
ltdt = dtxt(res.index[-1])['mon1']
seutot = df['seuninc'].iloc[-1] / 1_000
seush = res['uninc'].iloc[-1]
seu1yr = res['uninc'].iloc[-12:].mean()
seu19 = res.loc['2019', 'uninc'].mean()
seu89 = res.loc['1989':'1994', 'uninc'].mean()

text = (f'As of {ltdt}, there are {seutot:.1f} million \\textbf{{unincorporated '+
        f'self-employed}}, equivalent to {seush:.1f} percent of the labor '+
        f'force {c_line(colors["uninc"])}. Over the past year, the '+
        f'unincorporated self-employed made up an average of {seu1yr:.1f} '+
        f'percent of the labor force, compared to an average of {seu19:.1f} '+
        'percent in 2019. From 1989 to 1994, the category made up an '+
        f'average of {seu89:.1f} percent of the labor force. ')
write_txt(text_dir / 'selfemp1.txt', text)
print(text, '\n')

seitot = df['seinc'].iloc[-1] / 1_000
seish = res['inc'].iloc[-1]
sei19 = res.loc['2019', 'inc'].mean()
sei89 = res.loc['1989':'1994', 'INC'].mean()

text = (f'The \\textbf{{incorporated self-employed}} total {seitot:.1f} million '+
        f'in {ltdt}, equivalent to {seish:.1f} percent of the labor force '+
        f'{c_line(colors["inc"])}. In 2019, the category made up {sei19:.1f} percent '+
        'of the labor force.\n\nIncorporated self-employed are not reported by '+
        'BLS prior to 2000, but can be calculated from the CPS, and make up a '+
        f'average of {sei89:.1f} percent of the labor force from 1989 to 1994. ')
print(text)
write_txt(text_dir / 'selfemp2.txt', text)

As of December 2023, there are 10.0 million \textbf{unincorporated self-employed}, equivalent to 6.0 percent of the labor force (see {\color{purple}\textbf{---}}). Over the past year, the unincorporated self-employed made up an average of 5.8 percent of the labor force, compared to an average of 5.8 percent in 2019. From 1989 to 1994, the category made up an average of 8.0 percent of the labor force.  

The \textbf{incorporated self-employed} total 6.6 million in December 2023, equivalent to 4.0 percent of the labor force (see {\color{green!80!black}\textbf{---}}). In 2019, the category made up 3.8 percent of the labor force.

Incorporated self-employed are not reported by BLS prior to 2000, but can be calculated from the CPS, and make up a average of 2.8 percent of the labor force from 1989 to 1994. 


### Flows, Newly Employed, Not looking for work previously

In [16]:
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', 
                  parse_dates=['date'])
        .set_index('date')).loc['1990':, ['NILF', 'UNEMP']]
df['TOTAL'] = df.astype('float').sum(axis=1)
sh = (df['NILF'] / df['TOTAL']).rename('total') * 100

sh.to_csv(data_dir / 'lf_flow.csv', index_label='date', 
          header=True, float_format='%g')
ma = sh.resample('QS').mean().rename('quarterly')
ma.to_csv(data_dir / 'lf_flow_q.csv', index_label='date', 
          header=True, float_format='%g')

col = 'green!60!teal!80!black'
col2 = 'lime!80!green'
node = end_node(ma, col, date='q', size=1.2)
write_txt(text_dir / 'lf_flow_node.txt', node)

totval = df['TOTAL'].iloc[-1] / 1000
shval = sh.iloc[-1]
maval = ma.iloc[-1] 
sh4y = sh.iloc[-49]

ltdt = dtxt(sh.index[-1])['mon1']
prdt = dtxt(sh.index[-49])['mon1']

text = (f'In {ltdt}, {totval:.1f} million people were newly '+
        f'employed (on a gross basis). Of these, {shval:.1f} '+
        f'percent were not looking for work in the prior month '+
        f'{c_line(col2)}. Over the past three months, an average '+
        f'of {maval:.1f} percent of the newly employed were not '+
        f'looking for work the month prior {c_line(col)}.\n\nWhen '+
        'unemployment is low, the newly employed are more '+
        'likely to come from outside of the labor force. Four '+
        f'years ago, in {prdt}, {sh4y:.1f} percent of the newly '+
        'employed had not looked for work the previous month.')
write_txt(text_dir / 'lf_flow.txt', text)
print(text)

In December 2023, 5.6 million people were newly employed (on a gross basis). Of these, 69.3 percent were not looking for work in the prior month (see {\color{lime!80!green}\textbf{---}}). Over the past three months, an average of 70.9 percent of the newly employed were not looking for work the month prior (see {\color{green!60!teal!80!black}\textbf{---}}).

When unemployment is low, the newly employed are more likely to come from outside of the labor force. Four years ago, in December 2019, 72.6 percent of the newly employed had not looked for work the previous month.
